In [23]:
from sklearn.datasets import load_boston 
from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import StandardScaler

import numpy as np

import tensorflow as tf
from tensorflow import keras

### Preparing the Data

In [24]:
housing = load_boston()

In [25]:
X_train_full, X_test, y_train_full, y_test = train_test_split( housing.data, housing.target)
X_train, X_valid, y_train, y_valid = train_test_split( X_train_full, y_train_full)

print((X_train_full[0,:]))

print((X_train_full[0, np.newaxis]))
print((y_train_full[0]))

[3.6590e-02 2.5000e+01 4.8600e+00 0.0000e+00 4.2600e-01 6.3020e+00
 3.2200e+01 5.4007e+00 4.0000e+00 2.8100e+02 1.9000e+01 3.9690e+02
 6.7200e+00]
[[3.6590e-02 2.5000e+01 4.8600e+00 0.0000e+00 4.2600e-01 6.3020e+00
  3.2200e+01 5.4007e+00 4.0000e+00 2.8100e+02 1.9000e+01 3.9690e+02
  6.7200e+00]]
24.8


In [26]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train) 
X_valid = scaler.transform(X_valid) 
X_test = scaler.transform(X_test)

In [27]:
print("The input shape:")
print(X_train.shape[1:])

The input shape:
(13,)


### Build the Model

In [28]:
input_ = keras.layers.Input(shape=X_train.shape[1:])
hidden1 = keras.layers.Dense(30, activation="relu")(input_) 
hidden2 = keras.layers.Dense(30, activation="relu")(hidden1) 
concat = keras.layers.Concatenate()([input_, hidden2]) 
output = keras.layers.Dense(1)(concat)
model = keras.Model(inputs=[input_], outputs=[output])

model.summary()

Model: "functional_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 13)]         0                                            
__________________________________________________________________________________________________
dense_6 (Dense)                 (None, 30)           420         input_3[0][0]                    
__________________________________________________________________________________________________
dense_7 (Dense)                 (None, 30)           930         dense_6[0][0]                    
__________________________________________________________________________________________________
concatenate_2 (Concatenate)     (None, 43)           0           input_3[0][0]                    
                                                                 dense_7[0][0]         

### Compile the Model

In [29]:
model.compile(loss="mean_squared_error", optimizer="sgd")

### Fit the Model

In [30]:
early_stopping_callback = keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)

history = model.fit(X_train, y_train, epochs=40, 
                    validation_data=(X_valid, y_valid),
                   callbacks=[early_stopping_callback])

Epoch 1/40
9/9 [==============================] - 0s 20ms/step - loss: 486.5520 - val_loss: 309.0211
Epoch 2/40
9/9 [==============================] - 0s 5ms/step - loss: 869.3998 - val_loss: 418.9066
Epoch 3/40
9/9 [==============================] - 0s 5ms/step - loss: 474.2013 - val_loss: 208.2141
Epoch 4/40
9/9 [==============================] - 0s 5ms/step - loss: 142.5111 - val_loss: 108.4816
Epoch 5/40
9/9 [==============================] - 0s 4ms/step - loss: 104.7685 - val_loss: 121.2771
Epoch 6/40
9/9 [==============================] - 0s 6ms/step - loss: 108.0542 - val_loss: 107.3499
Epoch 7/40
9/9 [==============================] - 0s 8ms/step - loss: 48.3231 - val_loss: 58.7178
Epoch 8/40
9/9 [==============================] - 0s 11ms/step - loss: 40.8315 - val_loss: 45.8273
Epoch 9/40
9/9 [==============================] - 0s 14ms/step - loss: 23.6098 - val_loss: 44.1043
Epoch 10/40
9/9 [==============================] - 0s 9ms/step - loss: 22.9797 - val_loss: 44.3839
Epoc

### Evaluate the Model with the test data

In [31]:
mse_test = model.evaluate(X_test, y_test)

4/4 [==============================] - 0s 2ms/step - loss: 9.8348


### Use the model for prediction

In [32]:
X_new = X_test[:5] # pretend these are new instances 
y_pred = model.predict(X_new)
print(y_pred)
print(y_test[:5])

[[27.389896]
 [12.993153]
 [21.68355 ]
 [35.075184]
 [38.323353]]
[32.5 12.  24.7 33.4 41.3]


In [33]:
X_new = X_test[0, np.newaxis]
print(X_new)
y_pred = model.predict(X_new)
print(y_pred)
print(y_test[0])

[[-0.42788382 -0.47880263 -1.30255092 -0.26013299 -0.61253132  0.43584159
   0.91158264 -0.4548671  -0.79846364 -1.29842921 -0.34955742  0.47441695
  -1.01791159]]
[[27.389898]]
32.5
